In [262]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, FunctionTransformer
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
import optuna
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Reading the files


In [263]:
df_test = pd.read_csv("Test.csv")
df_train = pd.read_csv("Train.csv")
df_gap_test = pd.read_csv("Gap_test.csv")
df_gap_train = pd.read_csv("Gap_train.csv")
df_sample_submission = pd.read_csv("SampleSubmission.csv")
landsat_8 = pd.read_csv("LANDSAT8_data_updated.csv")

In [264]:
df_train.head(5)

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,dows,ecec20,hp20,ls,lstd,lstn,mb1,mb2,mb3,mb7,mdem,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity,N,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,0.581573,22.00,21.500278,11.007790,0.00779,0.030000,44.908058,18.967873,2006.000488,3182.000732,855.000244,2363.000732,1097,20.544283,126.835480,7.050,1.962921,39.00,9.75,7.962668,0.016853,0.000708,1.46,1300,0.34,147,6830,2310,5.66,75.2,85.0,0.82,2.98,0.24
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,0.707011,24.00,21.389599,14.023500,0.02350,0.030000,44.985626,19.730261,1637.000122,2839.000488,707.000061,2039.000488,1060,18.869566,109.835541,6.975,0.162065,40.00,8.00,8.439500,0.018321,0.001676,1.52,1400,11.70,151,1180,235,19.40,96.2,409.0,2.57,4.32,0.10
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,0.362439,15.25,18.900057,16.062401,0.06240,0.030000,44.167717,19.413284,1639.999634,2903.000000,758.999939,2003.999878,1074,24.719807,214.385269,6.725,0.744845,46.00,9.25,8.289246,0.020588,0.003885,1.46,3500,21.80,151,1890,344,11.00,76.7,65.0,1.95,1.24,0.22
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,0.531739,22.00,17.022963,18.030899,0.03090,0.030000,43.281063,19.539835,1325.000122,2413.000244,631.999939,1961.000000,1044,27.230274,255.713043,6.625,0.708708,43.75,10.00,8.666523,0.016913,0.001714,1.48,2300,39.90,201,6660,719,14.90,81.9,73.0,4.90,3.08,0.87
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,0.039202,14.75,23.103102,11.000000,0.00000,0.155324,45.654484,18.690720,1628.999512,2685.999023,732.999939,2427.000000,1055,20.434782,86.220909,6.700,0.634153,49.25,7.00,15.139549,0.019791,0.000000,1.43,940,1.00,90,7340,1160,8.66,69.4,149.0,0.55,3.03,0.31


In [265]:
df_test.head(5)

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,dows,ecec20,hp20,ls,lstd,lstn,mb1,mb2,mb3,mb7,mdem,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity
0,site_id_hgJpkz,ID_NGS9Bx,69.170794,44.522885,6.86,144,256,910,108,186,0.038132,44.75,1.581062,17.099701,0.0997,0.03,40.674942,18.333445,991.000061,2484.000488,486.000031,1413.000244,817,37.940418,467.619293,6.825,1.056416,25.50,15.25,8.732471,0.016981,0.005831,1.20
1,site_id_olmuI5,ID_YdVKXw,68.885265,44.741057,7.08,129,260,851,110,187,0.040187,53.00,5.700576,25.129000,0.1290,0.03,41.836205,18.762138,819.000000,2257.000000,412.000000,1199.000122,759,35.961353,542.590149,6.725,0.730379,18.75,14.00,10.565657,0.021030,0.005134,1.24
2,site_id_PTZdJz,ID_MZAlfE,68.970210,44.675777,6.50,142,259,901,109,187,0.068795,49.00,6.823292,26.117001,0.1170,0.03,42.058319,18.344410,965.000000,2378.000244,444.999969,1324.000000,769,38.983898,416.385437,6.825,1.146542,21.00,14.00,9.590125,0.018507,0.004480,1.23
3,site_id_DOTgr8,ID_GwCCMN,69.068751,44.647707,6.82,142,261,847,109,187,0.089460,46.25,2.235947,20.136999,0.1370,0.03,40.605465,18.348402,909.000122,2477.000000,471.000061,1300.000122,742,39.948471,374.971008,6.725,0.567210,23.25,12.25,9.669279,0.021688,0.006803,1.22
4,site_id_1rQNvy,ID_K8sowf,68.990002,44.577607,6.52,145,253,1109,110,186,0.066978,44.75,9.803858,28.024500,0.0245,0.03,40.671932,18.954443,1057.999390,2782.997559,461.999969,1292.999878,881,33.658615,361.233643,6.200,1.169207,26.25,18.25,7.895920,0.023016,0.000874,1.23


In [266]:
df_gap_train.head(5)

,Nutrient,Required,Available,Gap,PID
0,N,100.0,3796.0000,-3696.0000,ID_I5RGjv
1,P,40.0,0.9928,39.0072,ID_I5RGjv
2,K,52.0,429.2400,-377.2400,ID_I5RGjv
3,Ca,12.0,19943.6000,-19931.6000,ID_I5RGjv
4,Mg,8.0,6745.2000,-6737.2000,ID_I5RGjv


In [267]:
df_gap_test.head(5)

,Nutrient,Required,PID
0,N,100.0,ID_NGS9Bx
1,P,40.0,ID_NGS9Bx
2,K,52.0,ID_NGS9Bx
3,Ca,12.0,ID_NGS9Bx
4,Mg,8.0,ID_NGS9Bx


In [268]:
df_sample_submission.head(5)

,ID,Gap
0,ID_002W8m_B,0
1,ID_002W8m_Ca,0
2,ID_002W8m_Cu,0
3,ID_002W8m_Fe,0
4,ID_002W8m_K,0


In [269]:
landsat_8.head(5)

,QA_PIXEL,QA_RADSAT,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,ST_B10,date,lat,lon,PID
0,21824,0,0.035153,0.047720,0.080417,0.109732,0.163467,0.185798,0.145070,1.186770,2017-01-06,45.717723,70.451092,ID_7aZMWw
1,21824,0,0.029323,0.040075,0.068537,0.090180,0.150570,0.157335,0.122987,1.187870,2017-01-06,45.717725,70.451370,ID_7aZMWw
2,21824,0,0.041752,0.055365,0.086303,0.113280,0.184202,0.189263,0.145455,1.183498,2017-01-06,45.717452,70.451055,ID_7aZMWw
3,21824,0,0.047363,0.064000,0.106185,0.142540,0.208540,0.221988,0.161295,1.170078,2017-01-06,45.750596,70.516351,ID_BieBEw
4,21824,0,0.052808,0.072717,0.116443,0.159562,0.235380,0.250340,0.178565,1.163368,2017-01-06,45.750597,70.516628,ID_BieBEw


In [270]:
numeric_cols = landsat_8.columns.difference(['date', 'lat', 'lon'])

landsat_agg = landsat_8.groupby('PID')[numeric_cols].mean().reset_index()

def compute_landsat_indices(data_frame):
    df = data_frame.copy()

    # Convert bands to float
    bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6"]
    for b in bands:
        df[b] = df[b].astype(float)

    # NDVI
    df['NDVI'] = (df['SR_B4'] - df['SR_B3']) / (df['SR_B4'] + df['SR_B3'])

    # NDWI McFeeters
    df['NDWI_mcf'] = (df['SR_B2'] - df['SR_B4']) / (df['SR_B2'] + df['SR_B4'])

    # NDWI Gao
    df['NDWI_gao'] = (df['SR_B4'] - df['SR_B5']) / (df['SR_B4'] + df['SR_B5'])

    # NDRE (approximate with SWIR1)
    df['NDRE_approx'] = (df['SR_B4'] - df['SR_B5']) / (df['SR_B4'] + df['SR_B5'])

    # EVI
    df["EVI"] = 2.5 * (df["SR_B4"] - df["SR_B3"]) / (
        df["SR_B4"] + 6 * df["SR_B3"] - 7.5 * df["SR_B1"] + 1
    )

    # SAVI
    L = 0.5
    df["SAVI"] = ((df["SR_B4"] - df["SR_B3"]) * (1 + L)) / (
        df["SR_B4"] + df["SR_B3"] + L
    )

    # NBR
    df["NBR"] = (df["SR_B4"] - df["SR_B6"]) / (df["SR_B4"] + df["SR_B6"])

    return df

landsat_indices = compute_landsat_indices(landsat_agg)
landsat_indices.columns

Index(['PID', 'QA_PIXEL', 'QA_RADSAT', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4',
       'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'NDVI', 'NDWI_mcf', 'NDWI_gao',
       'NDRE_approx', 'EVI', 'SAVI', 'NBR'],
      dtype='object')

In [271]:
df_train = pd.merge(df_train ,landsat_indices , on = ['PID'] ,how = 'left')
df_test = pd.merge(df_test ,landsat_indices , on = ['PID'] ,how = 'left')

In [272]:
df_train

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,dows,ecec20,hp20,ls,lstd,lstn,mb1,mb2,mb3,mb7,mdem,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity,N,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B,QA_PIXEL,QA_RADSAT,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,ST_B10,NDVI,NDWI_mcf,NDWI_gao,NDRE_approx,EVI,SAVI,NBR
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,0.581573,22.00,21.500278,11.007790,0.00779,0.030000,44.908058,18.967873,2006.000488,3182.000732,855.000244,2363.000732,1097,20.544283,126.835480,7.050,1.962921,39.00,9.75,7.962668,0.016853,0.000708,1.46,1300,0.34,147,6830,2310,5.66,75.2,85.0,0.82,2.98,0.24,23361.253731,0.0,0.106083,0.128771,0.184369,0.229442,0.356474,0.374687,0.237561,1.076409,0.108923,-0.281036,-0.216809,-0.216809,0.073170,0.073987,-0.240420
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,0.707011,24.00,21.389599,14.023500,0.02350,0.030000,44.985626,19.730261,1637.000122,2839.000488,707.000061,2039.000488,1060,18.869566,109.835541,6.975,0.162065,40.00,8.00,8.439500,0.018321,0.001676,1.52,1400,11.70,151,1180,235,19.40,96.2,409.0,2.57,4.32,0.10,23937.900498,0.0,0.086580,0.104846,0.152806,0.193975,0.300232,0.298544,0.214672,1.103401,0.118717,-0.298271,-0.215004,-0.215004,0.070424,0.072927,-0.212315
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,0.362439,15.25,18.900057,16.062401,0.06240,0.030000,44.167717,19.413284,1639.999634,2903.000000,758.999939,2003.999878,1074,24.719807,214.385269,6.725,0.744845,46.00,9.25,8.289246,0.020588,0.003885,1.46,3500,21.80,151,1890,344,11.00,76.7,65.0,1.95,1.24,0.22,24046.995025,0.0,0.088759,0.108226,0.162777,0.204417,0.323351,0.305521,0.209319,1.086374,0.113400,-0.307671,-0.225352,-0.225352,0.068695,0.072025,-0.198267
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,0.531739,22.00,17.022963,18.030899,0.03090,0.030000,43.281063,19.539835,1325.000122,2413.000244,631.999939,1961.000000,1044,27.230274,255.713043,6.625,0.708708,43.75,10.00,8.666523,0.016913,0.001714,1.48,2300,39.90,201,6660,719,14.90,81.9,73.0,4.90,3.08,0.87,25109.044776,0.0,0.076020,0.096302,0.145362,0.178763,0.289124,0.290404,0.217235,1.103176,0.103048,-0.299789,-0.235871,-0.235871,0.056390,0.060793,-0.237955
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,0.039202,14.75,23.103102,11.000000,0.00000,0.155324,45.654484,18.690720,1628.999512,2685.999023,732.999939,2427.000000,1055,20.434782,86.220909,6.700,0.634153,49.25,7.00,15.139549,0.019791,0.000000,1.43,940,1.00,90,7340,1160,8.66,69.4,149.0,0.55,3.03,0.31,24045.104478,0.0,0.097326,0.114062,0.168237,0.206921,0.301310,0.326746,0.252330,1.109940,0.103115,-0.289297,-0.185719,-0.185719,0.065064,0.066304,-0.224530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7739,site_id_qnYCGg,ID_SL3ETR,70.240079,39.738497,5.40,140,143,1453,66,141,0.945865,26.50,6.670510,18.969999,2.97000,0.030000,27.021521,7.783801,976.999817,2827.999756,642.000000,1369.000000,2650,43.584541,183.406494,5.775,1.652288,32.00,37.25,7.895906,0.017060,0.156563,1.11,1900,2.00,522,1878,305,11.00,124.0,185.0,4.60,2.00,0.39,24826.112994,0.0,0.185279,0.195581,0.223813,0.231943,0.391283,0.274728,0.203960,0.933066,0.017838,-0.085052,-0.255670,-0.255670,0.017148,0.012759,-0.084444
7740,site_id_qnYCGg,ID_m3eydu,70.241244,39.735097,5.50,141,143,1453,66,141,0.067313,27.00,7.158565,21.340000,4.34000,0.030000,27.021521,7.783801,922.999878,2862.000244,637.999939,1375.000000,2646,43.584541,183.406494,5.525,9.693974,31.00,36.75,6.421572,0.016046,0.203374,1.12,1700,2.00,264,1317,211,12.00,122.0,217.0,5.30,2.50,0.20,25275.932203,0.0,0.208186,0.216478,0.238102,0.241586,0.405465,0.258042,0.192614,0.925893,0.007263,-0.054815,-0.253269,-0.253269,0.007856,0.005335,-0.032935
7741,site_id_qnYCGg,ID_GGQ39P,70.22

# Feature Engineering


In [273]:
target = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
uncessessary_columns = ['site', 'PID', 'lat', 'lon', 'pH', 'ph20']

In [274]:
# Check for duplicate values
df_train.duplicated().sum()

0

In [275]:
# check for missing values
df_train.isnull().sum()

site           0
PID            0
lon            0
lat            0
pH             0
              ..
NDWI_gao       3
NDRE_approx    3
EVI            3
SAVI           3
NBR            3
Length: 61, dtype: int64

In [276]:
# Fill missing values with the mean for columns with missing values in train_df
for column in df_train.columns:
  if df_train[column].isnull().any():
    df_train[column].fillna(df_train[column].mean(), inplace=True)

# Fill missing values with the mean for columns with missing values in test_df
for column in df_test.columns:
  if df_test[column].isnull().any():
    df_test[column].fillna(df_test[column].mean(), inplace=True)

In [277]:
# check for missing values
df_train.isnull().sum()

site           0
PID            0
lon            0
lat            0
pH             0
              ..
NDWI_gao       0
NDRE_approx    0
EVI            0
SAVI           0
NBR            0
Length: 61, dtype: int64

In [278]:
df_train.columns

Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
       'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
       'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
       'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity', 'N', 'P', 'K',
       'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B', 'QA_PIXEL', 'QA_RADSAT',
       'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',
       'NDVI', 'NDWI_mcf', 'NDWI_gao', 'NDRE_approx', 'EVI', 'SAVI', 'NBR'],
      dtype='object')

In [279]:
# function to convert ph to categorical
def convert_ph_to_categorical(ph):
    if ph <= 2.5:
        return 'strong_acid'
    elif 2.5 <= ph < 6.5:
        return 'weak_acid'
    elif 6.5 <= ph < 7.5:
        return 'neutral'
    elif 7.5 <= ph < 10.5:
        return 'weak_alkaline'
    else:
        return 'strong_alkaline'
    
df_train['pH_category'] = df_train['pH'].apply(convert_ph_to_categorical)
df_test['pH_category'] = df_test['pH'].apply(convert_ph_to_categorical)
df_train['ph20_category'] = df_train['ph20'].apply(convert_ph_to_categorical)
df_test['ph20_category'] = df_test['ph20'].apply(convert_ph_to_categorical)

In [280]:
# Site means
def group_by_site_and_calculate_mean(df):
    # Exclude 'lat' and 'lon' from numeric columns
    numeric_columns = df.columns.difference(target + ['lat', 'lon', 'PID'])
    
    # Group by 'site' and calculate the mean for the selected columns
    grouped_df = df.groupby('site')[numeric_columns].mean().reset_index()
    grouped_df.columns = ['site'] + [f"{col}_site_mean" for col in grouped_df.columns if col != 'site']
    
    return grouped_df

df_train_site_mean = group_by_site_and_calculate_mean(df_train)
df_test_site_mean = group_by_site_and_calculate_mean(df_test)
# Merge the mean values back to the original dataframes
df_train = df_train.merge(df_train_site_mean, on='site', how='left')
df_test = df_test.merge(df_test_site_mean, on='site', how='left')

In [281]:
# Pid means
def group_by_pid_and_calculate_mean(df):
    # Exclude 'lat' and 'lon' from numeric columns
    numeric_columns = df.columns.difference(target + ['lat', 'lon', 'site'])
    
    # Group by 'site' and calculate the mean for the selected columns
    grouped_df = df.groupby('PID')[numeric_columns].mean().reset_index()
    grouped_df.columns = ['PID'] + [f"{col}_pid_mean" for col in grouped_df.columns if col != 'PID']
    
    return grouped_df

df_train_pid_mean = group_by_pid_and_calculate_mean(df_train)
df_test_pid_mean = group_by_pid_and_calculate_mean(df_test)
# Merge the mean values back to the original dataframes
df_train = df_train.merge(df_train_pid_mean, on='PID', how='left')
df_test = df_test.merge(df_test_pid_mean, on='PID', how='left')

In [282]:
# Function 1: Soil chemical and physical ratios
def add_soil_chemical_ratios(df):
    df = df.copy()
    with np.errstate(divide='ignore', invalid='ignore'):
        df['CEC_to_ECEC'] = df['cec20'] / df['ecec20']
        df['SOC_to_BD'] = df['soc20'] / df['BulkDensity']
        df['xhp_to_ph'] = df['xhp20'] / df['ph20']
    return df

df_train = add_soil_chemical_ratios(df_train)
df_test = add_soil_chemical_ratios(df_test)

In [283]:
# Function 2: Elevation and geospatial features

def add_geospatial_features(df, n_clusters=5):
    df = df.copy()
    # Cluster based on lat and lon
    coords = df[['lat', 'lon']].dropna()
    scaler = StandardScaler()
    coords_scaled = scaler.fit_transform(coords)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    clusters = kmeans.fit_predict(coords_scaled)
    df.loc[coords.index, 'location_cluster'] = clusters

    # Distance from centroid
    center_lat, center_lon = df['lat'].mean(), df['lon'].mean()
    df['distance_from_center'] = np.sqrt((df['lat'] - center_lat) ** 2 + (df['lon'] - center_lon) ** 2)

    return df

df_train = add_geospatial_features(df_train)
df_test = add_geospatial_features(df_test)

In [284]:
# function 3: Vegetation indices
def add_vegetation_features(df):
    df = df.copy()
    # Interaction terms
    df['NDVI_SAVI'] = df['NDVI'] * df['SAVI']
    df['NDVI_to_NDRE'] = df['NDVI'] / df['NDRE_approx'].replace(0, np.nan)
    df['NDWI_diff'] = df['NDWI_mcf'] - df['NDWI_gao']
    df['bio1_to_bio12'] = df['bio1'] / df['bio12'].replace(0, np.nan)
    df['bio7_bio15'] = df['bio7'] * df['bio15']
    df['bio1_bio7'] = df['bio1'] * df['bio7']

    # PCA on vegetation indices
    veg_features = ['NDVI', 'SAVI', 'EVI', 'NDRE_approx', 'NDWI_mcf', 'NDWI_gao']
    pca_input = df[veg_features].fillna(0)
    pca = PCA(n_components=2)
    pca_components = pca.fit_transform(pca_input)
    df['veg_pca1'] = pca_components[:, 0]
    df['veg_pca2'] = pca_components[:, 1]

    return df

df_train = add_vegetation_features(df_train)
df_test = add_vegetation_features(df_test)

In [285]:
# Function 4: Remote sensing band ratios and PCA
def add_remote_sensing_features(df):
    df = df.copy()
    # Band ratios
    df['B4_to_B3'] = df['SR_B4'] / df['SR_B3'].replace(0, np.nan)
    df['B5_to_B4'] = df['SR_B5'] / df['SR_B4'].replace(0, np.nan)

    # PCA on bands
    band_features = [f'SR_B{i}' for i in range(1, 8)] + ['ST_B10']
    band_input = df[band_features].fillna(0)
    pca = PCA(n_components=2)
    band_components = pca.fit_transform(band_input)
    df['band_pca1'] = band_components[:, 0]
    df['band_pca2'] = band_components[:, 1]

    return df

df_train = add_remote_sensing_features(df_train)
df_test = add_remote_sensing_features(df_test)


In [286]:
df_train

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,dows,ecec20,hp20,ls,lstd,lstn,mb1,mb2,mb3,mb7,mdem,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity,N,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B,QA_PIXEL,QA_RADSAT,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,ST_B10,NDVI,NDWI_mcf,NDWI_gao,NDRE_approx,EVI,SAVI,NBR,pH_category,ph20_category,BulkDensity_site_mean,EVI_site_mean,NBR_site_mean,NDRE_approx_site_mean,NDVI_site_mean,NDWI_gao_site_mean,NDWI_mcf_site_mean,QA_PIXEL_site_mean,QA_RADSAT_site_mean,SAVI_site_mean,SR_B1_site_mean,SR_B2_site_mean,SR_B3_site_mean,SR_B4_site_mean,SR_B5_site_mean,SR_B6_site_mean,SR_B7_site_mean,ST_B10_site_mean,alb_site_mean,bio1_site_mean,bio12_site_mean,bio15_site_mean,bio7_site_mean,bp_site_mean,cec20_site_mean,dows_site_mean,ecec20_site_mean,hp20_site_mean,ls_site_mean,lstd_site_mean,lstn_site_mean,mb1_site_mean,mb2_site_mean,mb3_site_mean,mb7_site_mean,mdem_site_mean,pH_site_mean,para_site_mean,parv_site_mean,ph20_site_mean,slope_site_mean,snd20_site_mean,soc20_site_mean,tim_site_mean,wp_site_mean,xhp20_site_mean,BulkDensity_pid_mean,BulkDensity_site_mean_pid_mean,EVI_pid_mean,EVI_site_mean_pid_mean,NBR_pid_mean,NBR_site_mean_pid_mean,NDRE_approx_pid_mean,NDRE_approx_site_mean_pid_mean,NDVI_pid_mean,NDVI_site_mean_pid_mean,NDWI_gao_pid_mean,NDWI_gao_site_mean_pid_mean,NDWI_mcf_pid_mean,NDWI_mcf_site_mean_pid_mean,QA_PIXEL_pid_mean,QA_PIXEL_site_mean_pid_mean,QA_RADSAT_pid_mean,QA_RADSAT_site_mean_pid_mean,SAVI_pid_mean,SAVI_site_mean_pid_mean,SR_B1_pid_mean,SR_B1_site_mean_pid_mean,SR_B2_pid_mean,SR_B2_site_mean_pid_mean,SR_B3_pid_mean,SR_B3_site_mean_pid_mean,SR_B4_pid_mean,SR_B4_site_mean_pid_mean,SR_B5_pid_mean,SR_B5_site_mean_pid_mean,SR_B6_pid_mean,SR_B6_site_mean_pid_mean,SR_B7_pid_mean,SR_B7_site_mean_pid_mean,ST_B10_pid_mean,ST_B10_site_mean_pid_mean,alb_pid_mean,alb_site_mean_pid_mean,bio1_pid_mean,bio12_pid_mean,bio12_site_mean_pid_mean,bio15_pid_mean,bio15_site_mean_pid_mean,bio1_site_mean_pid_mean,bio7_pid_mean,bio7_site_mean_pid_mean,bp_pid_mean,bp_site_mean_pid_mean,cec20_pid_mean,cec20_site_mean_pid_mean,dows_pid_mean,dows_site_mean_pid_mean,ecec20_pid_mean,ecec20_site_mean_pid_mean,hp20_pid_mean,hp20_site_mean_pid_mean,ls_pid_mean,ls_site_mean_pid_mean,lstd_pid_mean,lstd_site_mean_pid_mean,lstn_pid_mean,lstn_site_mean_pid_mean,mb1_pid_mean,mb1_site_mean_pid_mean,mb2_pid_mean,mb2_site_mean_pid_mean,mb3_pid_mean,mb3_site_mean_pid_mean,mb7_pid_mean,mb7_site_mean_pid_mean,mdem_pid_mean,mdem_site_mean_pid_mean,pH_pid_mean,pH_site_mean_pid_mean,para_pid_mean,para_site_mean_pid_mean,parv_pid_mean,parv_site_mean_pid_mean,ph20_pid_mean,ph20_site_mean_pid_mean,slope_pid_mean,slope_site_mean_pid_mean,snd20_pid_mean,snd20_site_mean_pid_mean,soc20_pid_mean,soc20_site_mean_pid_mean,tim_pid_mean,tim_site_mean_pid_mean,wp_pid_mean,wp_site_mean_pid_mean,xhp20_pid_mean,xhp20_site_mean_pid_mean,CEC_to_ECEC,SOC_to_BD,xhp_to_ph,location_cluster,distance_from_center,NDVI_SAVI,NDVI_to_NDRE,NDWI_diff,bio1_to_bio12,bio7_bio15,bio1_bio7,veg_pca1,veg_pca2,B4_to_B3,B5_to_B4,band_pca1,band_pca2
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,0.581573,22.00,21.500278,11.007790,0.00779,0.030000,44.908058,18.967873,2006.000488,3182.000732,855.000244,2363.000732,1097,20.544283,126.835480,7.050,1.962921,39.00,9.75,7.962668,0.016853,0.000708,1.46,1300,0.34,147,6830,2310,5.66,75.2,85.0,0.82,2.98,0.24,23361.253731,0.0,0.106083,0.128771,0.184369,0.229442,0.356474,0.374687,0.237561,1.076409,0.108923,-0.281036,-0.216809,-0.216809,0.073170,0.073987,-0.240420,weak_alkaline,neutral,1.447500,0.057474,-0.204487,-0.226550,0.095381,-0.226550,-0.252584,24150.745958,0.0,0.057808,0.091250,0.108073,0.149834,0.182346,0.288982,0.276862,0.197123,1.097181,156.875000,248.000000,955.250000,108.375000,190.250000,0.353755,29.468750,22.401946,18.853312,0.103311,0.045196,45.435076,18.949129,1320.500130,2351.375214,599.500076,1826.625031,1089.750000,7.185000,22.205113,167.595243,7.146875,1.009586,37.843750,9.468750,10.2

In [287]:
df_train.duplicated().sum()

0

In [288]:
# # plot lat and lon values
# plt.figure(figsize=(10, 6))
# for i in target:
#     plt.subplot(3, 4, target.index(i) + 1)
#     plt.title(i)
#     plt.scatter(df_train["lon"], df_train["lat"], c=df_train[i], cmap="viridis", s=10)
#     plt.colorbar()
# plt.tight_layout()

In [289]:
# # function scatter plot of all columns and target
# def scatter_plot(df, target, specific_target):
#     plt.figure(figsize=(20, 20))
#     all_columns = df.columns.tolist()
#     for i in all_columns:
#         if i not in uncessessary_columns + target:
#             plt.subplot(7, 5, all_columns.index(i) + 1)
#             plt.title(i)
#             plt.scatter(df[i], df[specific_target])
#             plt.colorbar()
#     plt.tight_layout()
#     plt.show()

In [290]:
# for i in target:
#     print(f"Target is: {i}")
#     scatter_plot(df_train, target, i)

In [291]:
# scatter plot of all columns K
# scatter_plot(df_train, target, "K")

In [292]:
# # droping Na values
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [293]:
(df_train.isin([np.inf, -np.inf]).sum()).to_frame().T  # Count per column

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,dows,ecec20,hp20,ls,lstd,lstn,mb1,mb2,mb3,mb7,mdem,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity,N,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B,QA_PIXEL,QA_RADSAT,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,ST_B10,NDVI,NDWI_mcf,NDWI_gao,NDRE_approx,EVI,SAVI,NBR,pH_category,ph20_category,BulkDensity_site_mean,EVI_site_mean,NBR_site_mean,NDRE_approx_site_mean,NDVI_site_mean,NDWI_gao_site_mean,NDWI_mcf_site_mean,QA_PIXEL_site_mean,QA_RADSAT_site_mean,SAVI_site_mean,SR_B1_site_mean,SR_B2_site_mean,SR_B3_site_mean,SR_B4_site_mean,SR_B5_site_mean,SR_B6_site_mean,SR_B7_site_mean,ST_B10_site_mean,alb_site_mean,bio1_site_mean,bio12_site_mean,bio15_site_mean,bio7_site_mean,bp_site_mean,cec20_site_mean,dows_site_mean,ecec20_site_mean,hp20_site_mean,ls_site_mean,lstd_site_mean,lstn_site_mean,mb1_site_mean,mb2_site_mean,mb3_site_mean,mb7_site_mean,mdem_site_mean,pH_site_mean,para_site_mean,parv_site_mean,ph20_site_mean,slope_site_mean,snd20_site_mean,soc20_site_mean,tim_site_mean,wp_site_mean,xhp20_site_mean,BulkDensity_pid_mean,BulkDensity_site_mean_pid_mean,EVI_pid_mean,EVI_site_mean_pid_mean,NBR_pid_mean,NBR_site_mean_pid_mean,NDRE_approx_pid_mean,NDRE_approx_site_mean_pid_mean,NDVI_pid_mean,NDVI_site_mean_pid_mean,NDWI_gao_pid_mean,NDWI_gao_site_mean_pid_mean,NDWI_mcf_pid_mean,NDWI_mcf_site_mean_pid_mean,QA_PIXEL_pid_mean,QA_PIXEL_site_mean_pid_mean,QA_RADSAT_pid_mean,QA_RADSAT_site_mean_pid_mean,SAVI_pid_mean,SAVI_site_mean_pid_mean,SR_B1_pid_mean,SR_B1_site_mean_pid_mean,SR_B2_pid_mean,SR_B2_site_mean_pid_mean,SR_B3_pid_mean,SR_B3_site_mean_pid_mean,SR_B4_pid_mean,SR_B4_site_mean_pid_mean,SR_B5_pid_mean,SR_B5_site_mean_pid_mean,SR_B6_pid_mean,SR_B6_site_mean_pid_mean,SR_B7_pid_mean,SR_B7_site_mean_pid_mean,ST_B10_pid_mean,ST_B10_site_mean_pid_mean,alb_pid_mean,alb_site_mean_pid_mean,bio1_pid_mean,bio12_pid_mean,bio12_site_mean_pid_mean,bio15_pid_mean,bio15_site_mean_pid_mean,bio1_site_mean_pid_mean,bio7_pid_mean,bio7_site_mean_pid_mean,bp_pid_mean,bp_site_mean_pid_mean,cec20_pid_mean,cec20_site_mean_pid_mean,dows_pid_mean,dows_site_mean_pid_mean,ecec20_pid_mean,ecec20_site_mean_pid_mean,hp20_pid_mean,hp20_site_mean_pid_mean,ls_pid_mean,ls_site_mean_pid_mean,lstd_pid_mean,lstd_site_mean_pid_mean,lstn_pid_mean,lstn_site_mean_pid_mean,mb1_pid_mean,mb1_site_mean_pid_mean,mb2_pid_mean,mb2_site_mean_pid_mean,mb3_pid_mean,mb3_site_mean_pid_mean,mb7_pid_mean,mb7_site_mean_pid_mean,mdem_pid_mean,mdem_site_mean_pid_mean,pH_pid_mean,pH_site_mean_pid_mean,para_pid_mean,para_site_mean_pid_mean,parv_pid_mean,parv_site_mean_pid_mean,ph20_pid_mean,ph20_site_mean_pid_mean,slope_pid_mean,slope_site_mean_pid_mean,snd20_pid_mean,snd20_site_mean_pid_mean,soc20_pid_mean,soc20_site_mean_pid_mean,tim_pid_mean,tim_site_mean_pid_mean,wp_pid_mean,wp_site_mean_pid_mean,xhp20_pid_mean,xhp20_site_mean_pid_mean,CEC_to_ECEC,SOC_to_BD,xhp_to_ph,location_cluster,distance_from_center,NDVI_SAVI,NDVI_to_NDRE,NDWI_diff,bio1_to_bio12,bio7_bio15,bio1_bio7,veg_pca1,veg_pca2,B4_to_B3,B5_to_B4,band_pca1,band_pca2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [294]:
(df_train.isna().sum() >= 0).to_frame().T

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,dows,ecec20,hp20,ls,lstd,lstn,mb1,mb2,mb3,mb7,mdem,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity,N,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B,QA_PIXEL,QA_RADSAT,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,ST_B10,NDVI,NDWI_mcf,NDWI_gao,NDRE_approx,EVI,SAVI,NBR,pH_category,ph20_category,BulkDensity_site_mean,EVI_site_mean,NBR_site_mean,NDRE_approx_site_mean,NDVI_site_mean,NDWI_gao_site_mean,NDWI_mcf_site_mean,QA_PIXEL_site_mean,QA_RADSAT_site_mean,SAVI_site_mean,SR_B1_site_mean,SR_B2_site_mean,SR_B3_site_mean,SR_B4_site_mean,SR_B5_site_mean,SR_B6_site_mean,SR_B7_site_mean,ST_B10_site_mean,alb_site_mean,bio1_site_mean,bio12_site_mean,bio15_site_mean,bio7_site_mean,bp_site_mean,cec20_site_mean,dows_site_mean,ecec20_site_mean,hp20_site_mean,ls_site_mean,lstd_site_mean,lstn_site_mean,mb1_site_mean,mb2_site_mean,mb3_site_mean,mb7_site_mean,mdem_site_mean,pH_site_mean,para_site_mean,parv_site_mean,ph20_site_mean,slope_site_mean,snd20_site_mean,soc20_site_mean,tim_site_mean,wp_site_mean,xhp20_site_mean,BulkDensity_pid_mean,BulkDensity_site_mean_pid_mean,EVI_pid_mean,EVI_site_mean_pid_mean,NBR_pid_mean,NBR_site_mean_pid_mean,NDRE_approx_pid_mean,NDRE_approx_site_mean_pid_mean,NDVI_pid_mean,NDVI_site_mean_pid_mean,NDWI_gao_pid_mean,NDWI_gao_site_mean_pid_mean,NDWI_mcf_pid_mean,NDWI_mcf_site_mean_pid_mean,QA_PIXEL_pid_mean,QA_PIXEL_site_mean_pid_mean,QA_RADSAT_pid_mean,QA_RADSAT_site_mean_pid_mean,SAVI_pid_mean,SAVI_site_mean_pid_mean,SR_B1_pid_mean,SR_B1_site_mean_pid_mean,SR_B2_pid_mean,SR_B2_site_mean_pid_mean,SR_B3_pid_mean,SR_B3_site_mean_pid_mean,SR_B4_pid_mean,SR_B4_site_mean_pid_mean,SR_B5_pid_mean,SR_B5_site_mean_pid_mean,SR_B6_pid_mean,SR_B6_site_mean_pid_mean,SR_B7_pid_mean,SR_B7_site_mean_pid_mean,ST_B10_pid_mean,ST_B10_site_mean_pid_mean,alb_pid_mean,alb_site_mean_pid_mean,bio1_pid_mean,bio12_pid_mean,bio12_site_mean_pid_mean,bio15_pid_mean,bio15_site_mean_pid_mean,bio1_site_mean_pid_mean,bio7_pid_mean,bio7_site_mean_pid_mean,bp_pid_mean,bp_site_mean_pid_mean,cec20_pid_mean,cec20_site_mean_pid_mean,dows_pid_mean,dows_site_mean_pid_mean,ecec20_pid_mean,ecec20_site_mean_pid_mean,hp20_pid_mean,hp20_site_mean_pid_mean,ls_pid_mean,ls_site_mean_pid_mean,lstd_pid_mean,lstd_site_mean_pid_mean,lstn_pid_mean,lstn_site_mean_pid_mean,mb1_pid_mean,mb1_site_mean_pid_mean,mb2_pid_mean,mb2_site_mean_pid_mean,mb3_pid_mean,mb3_site_mean_pid_mean,mb7_pid_mean,mb7_site_mean_pid_mean,mdem_pid_mean,mdem_site_mean_pid_mean,pH_pid_mean,pH_site_mean_pid_mean,para_pid_mean,para_site_mean_pid_mean,parv_pid_mean,parv_site_mean_pid_mean,ph20_pid_mean,ph20_site_mean_pid_mean,slope_pid_mean,slope_site_mean_pid_mean,snd20_pid_mean,snd20_site_mean_pid_mean,soc20_pid_mean,soc20_site_mean_pid_mean,tim_pid_mean,tim_site_mean_pid_mean,wp_pid_mean,wp_site_mean_pid_mean,xhp20_pid_mean,xhp20_site_mean_pid_mean,CEC_to_ECEC,SOC_to_BD,xhp_to_ph,location_cluster,distance_from_center,NDVI_SAVI,NDVI_to_NDRE,NDWI_diff,bio1_to_bio12,bio7_bio15,bio1_bio7,veg_pca1,veg_pca2,B4_to_B3,B5_to_B4,band_pca1,band_pca2
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,Tr

# Modelling


In [295]:
X = df_train.drop(columns=target + uncessessary_columns)
X_test = df_test.drop(columns=uncessessary_columns)
y = df_train[target]


In [296]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

In [297]:
def clean_numeric(X):
    X = pd.DataFrame(X)  # Ensure it's a DataFrame for safe ops
    X.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infs with NaN
    return X

In [298]:
# creating pipeline
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = ['location_cluster', 'pH_category', 'ph20_category']

numeric_transformer = Pipeline(steps=[
    ('cleaner', FunctionTransformer(clean_numeric, validate=False)),
    ('scaler', StandardScaler()),
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [299]:
# creating xgboost model with optuna

def objective(trial):
    # Define the hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 100, 2000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    gamma = trial.suggest_float('gamma', 0, 5)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)

    # Create the model with the suggested hyperparameters
    xgb_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', MultiOutputRegressor(XGBRegressor(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth,
                min_child_weight=min_child_weight,
                gamma=gamma,
                subsample=subsample,
                colsample_bytree=colsample_bytree
            ))) 
        ])

    score = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_root_mean_squared_error')

    return score.mean() * -1

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

In [301]:
study.best_params

{'n_estimators': 214,
 'learning_rate': 0.04036148152109534,
 'max_depth': 5,
 'min_child_weight': 9,
 'gamma': 3.3766471428429807,
 'subsample': 0.9758383958517547,
 'colsample_bytree': 0.548427803085979}

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
# Creating boosting model
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', MultiOutputRegressor(XGBRegressor(**study.best_params)))
                    ])

# training models and evaluating performance
xgb_model.fit(x_train, y_train)
xgb_pred = xgb_model.predict(x_valid)
xgb_rmse = root_mean_squared_error(y_valid, xgb_pred)
print(f"XGBoost RMSE: {xgb_rmse:.2f}")
xgb_mae = mean_absolute_error(y_valid, xgb_pred)
print(f"XGBoost MAE: {xgb_mae:.2f}")
xgb_r2 = r2_score(y_valid, xgb_pred)
print(f"XGBoost R2: {xgb_r2:.2f}")

XGBoost RMSE: 236.12
XGBoost MAE: 151.75
XGBoost R2: 0.41


# Submission File


In [306]:
# training the models with the whole dataset
xgb_model.fit(X, y)
test_predictions = xgb_model.predict(X_test)

In [307]:
# Split the predictions into separate columns
N_pred =  test_predictions[:, 0]  # Predictions for N
P_pred =  test_predictions[:, 1]  # Predictions for P
K_pred =  test_predictions[:, 2]  # Predictions for K
Ca_pred = test_predictions[:, 3]  # Predictions for Ca
Mg_pred = test_predictions[:, 4]  # Predictions for Mg
S_pred =  test_predictions[:, 5]  # Predictions for S
Fe_pred = test_predictions[:, 6]  # Predictions for Fe
Mn_pred = test_predictions[:, 7]  # Predictions for Mn
Zn_pred = test_predictions[:, 8]  # Predictions for Zn
Cu_pred = test_predictions[:, 9]  # Predictions for Cu
B_pred =  test_predictions[:, 10]  # Predictions for B

submission = pd.DataFrame({'PID': df_test['PID'], 'N': N_pred, 'P': P_pred, 'K': K_pred, 'Ca': Ca_pred, 'Mg': Mg_pred, 'S': S_pred, 'Fe': Fe_pred, 'Mn': Mn_pred, 'Zn': Zn_pred, 'Cu': Cu_pred, 'B': B_pred})
submission_melted = submission.melt(id_vars=['PID'], var_name='Nutrient', value_name='Available_Nutrients_in_ppm')
submission_melted = submission_melted.sort_values('PID')


nutrient_df = pd.merge(df_gap_test, submission_melted, on=['PID', 'Nutrient'], how='left')
nutrient_df = pd.merge(nutrient_df, df_test[['PID', 'BulkDensity']], on='PID', how='left')

soil_depth = 20  # cm

# Calculate the Available_Nutrients_in_kg_ha
nutrient_df['Available_Nutrients_in_kg_ha'] = (nutrient_df['Available_Nutrients_in_ppm']
                                               * soil_depth * nutrient_df['BulkDensity'] * 0.1)

nutrient_df["Gap"] = nutrient_df["Required"] - nutrient_df["Available_Nutrients_in_kg_ha"]

nutrient_df['ID'] = nutrient_df['PID'] + "_" + nutrient_df['Nutrient']
nutrient_df = nutrient_df[['ID', 'Gap']]
nutrient_df.head()

,ID,Gap
0,ID_NGS9Bx_N,-4486.727656
1,ID_NGS9Bx_P,11.089716
2,ID_NGS9Bx_K,-219.911211
3,ID_NGS9Bx_Ca,-14300.460000
4,ID_NGS9Bx_Mg,-4140.637187


In [308]:
nutrient_df.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv")

Submission file saved as submission.csv
